In [31]:
import os 

os.getcwd()

'/home/nittaak/661'

In [32]:
import os
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
from transformers import PreTrainedTokenizerFast, AutoTokenizer, AutoModelForCausalLM
import torch
from cosinesimcontained import get_cosine_sim_bert
import numpy as np
from transformers import BertTokenizer, BertModel

def load_data(data_dir, labels_dir):
    data_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.txt')])
    label_files = sorted([f for f in os.listdir(labels_dir) if f.endswith('.txt')])

    dataset = []

    for data_file, label_file in zip(data_files, label_files):
        with open(os.path.join(data_dir, data_file), 'r') as df, open(os.path.join(labels_dir, label_file), 'r') as lf:
            data = df.read()
            label = lf.read()

            dataset.append((data, label))  # Append as a tuple (data, label)

    return dataset

class TextDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.labels[idx]

        if self.transform:
            data = self.transform(data)

        return data, label

model = BertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

## Validation Data
dataset_betham_val = load_data('/home/nittaak/661/validation/Betham/OCR', '/home/nittaak/661/validation/Betham/GT/')
dataset_IAM_val = load_data('/home/nittaak/661/validation/IAM/OCR/','/home/nittaak/661/validation/IAM/GT/')

data_betham_val, labels_betham_val = zip(*dataset_betham_val)
data_IAM_val, labels_IAM_val = zip(*dataset_IAM_val)

## Test Data
dataset_betham_test = load_data('/home/nittaak/661/test/Betham/OCR/', '/home/nittaak/661/test/Betham/GT/')
dataset_IAM_test = load_data('/home/nittaak/661/test/IAM/OCR/','/home/nittaak/661/test/IAM/GT/')

data_betham_test, labels_betham_test = zip(*dataset_betham_test)
data_IAM_test, labels_IAM_test = zip(*dataset_IAM_test)

In [33]:
cosine_sim_val = []
count_val = 0
device = torch.device("cuda:0")
model.to(device)

for data, label in dataset_betham_val:
    if data is None or label is None: 
        continue
    if len(data) == 0 or len(label) == 0: 
        continue
    cosine_sim_val.append( get_cosine_sim_bert(data, label, tokenizer=tokenizer, model=model,device=device))
    count_val += 1


for data, label in dataset_IAM_val:
    if data is None or label is None: 
        continue
    if len(data) == 0 or len(label) == 0: 
        continue
    cosine_sim_val.append( get_cosine_sim_bert(data, label, tokenizer=tokenizer, model=model,device=device))
    count_val += 1

cosine_sim_val_avg = np.mean(cosine_sim_val[0])

In [34]:
print(f'cosine similarity validation average: {cosine_sim_val_avg}')

cosine similarity validation average: 0.7338921888396631


In [36]:
device = torch.device("cuda:0")
model.to(device)

cosine_sim_test = []
count_test = 0

for data, label in dataset_betham_test:
    if data is None or label is None: 
        continue
    if len(data) == 0 or len(label) == 0: 
        continue
    cosine_sim_test.append( get_cosine_sim_bert(data, label, tokenizer=tokenizer, model=model,device=device))

for data, label in dataset_IAM_test: 
    if data is None or label is None: 
        continue
    if len(data) == 0 or len(label) == 0: 
        continue
    cosine_sim_test.append( get_cosine_sim_bert(data, label, tokenizer=tokenizer, model=model,device=device))
    
cosine_sim_test_avg = np.mean(cosine_sim_test[0])

In [38]:
print(f'cosine similarity test average: {cosine_sim_test_avg}')

cosine similarity test average: 0.7383061489172741
